# Process data

## Process error-based metrics (MAE, RMSE, NSE) with window lengths 90
(together with model size and runtime)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

CUR_ABS_DIR = Path.cwd().resolve()
PROJ_DIR = (CUR_ABS_DIR / '../../../').resolve()
OUTPUT_DIR = (PROJ_DIR / 'swissrivernetwork/benchmark/outputs/ray_results/').resolve()
DUMP_DIR = (CUR_ABS_DIR / 'outputs/radar/').resolve()
STATIONS_DUMP_DIR = (CUR_ABS_DIR / 'outputs/stations/').resolve()

LOAD_LATEST_RESULTS = True

ISSUE_TAG = '<span style="color:red;">[issue]</span> '
INFO_TAG = '<span style="color:blue;">[info]</span> '
SUCCESS_TAG = '<span style="color:green;">[success]</span> '


def print(text=None):
    from IPython.display import display, HTML
    if text is None:
        display(HTML("<br>"))
        return
    text = str(text)
    display(HTML(text))


print(f'{INFO_TAG} Project directory: {PROJ_DIR}')
print(f'{INFO_TAG} Current directory: {CUR_ABS_DIR}')
print(f'{INFO_TAG} Output directory: {OUTPUT_DIR}')
print(f'{INFO_TAG} Dump directory: {DUMP_DIR}')

In [3]:
METRICS_LIST = ['RMSE', 'MAE', 'NSE']
STATS_LIST = ['Mean', 'Std', 'Median', 'Min', 'Max', 'CI95']

In [4]:
# Get Err 90 data, Model Size data, Runtime data

import pandas as pd
import os
import json


def read_results_from_csv(
        output_dir: Path, graph_name: str, method: str, filename_key: str = 'results', verbose: bool = False
) -> pd.DataFrame:
    file_path = os.path.join(output_dir, f'{graph_name}_{method}_{filename_key}.csv')
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if verbose:
            print(f"{SUCCESS_TAG} Loaded results from {file_path}.")
        return df
    else:
        if verbose:
            print(f"{INFO_TAG} No results found at {file_path}.")
        return pd.DataFrame()


def save_results_to_csv(
        df: pd.DataFrame, output_dir: Path, graph_name: str, method: str, filename_key: str = 'results',
        verbose: bool = True
) -> str:
    output_dir.mkdir(parents=True, exist_ok=True)
    file_path = os.path.join(output_dir, f'{graph_name}_{method}_{filename_key}.csv')
    df.to_csv(file_path, index=False)
    if verbose:
        print(f"{SUCCESS_TAG} Saved results to {file_path}.")
    return file_path


def read_results_from_json(
        output_dir: Path, graph_name: str, method: str, filename_key: str = 'results', verbose: bool = False
) -> dict:
    file_path = os.path.join(output_dir, f'{graph_name}_{method}_{filename_key}.json')
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)
        if verbose:
            print(f"{SUCCESS_TAG} Loaded extra results from {file_path}.")
        return data
    else:
        if verbose:
            print(f"{INFO_TAG} No extra results found at {file_path}.")
        return {}


def save_results_to_json(
        data: dict, output_dir: Path, graph_name: str, method: str, filename_key: str = 'results',
        verbose: bool = True
) -> str:
    output_dir.mkdir(parents=True, exist_ok=True)
    file_path = os.path.join(output_dir, f'{graph_name}_{method}_{filename_key}.json')
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)
    if verbose:
        print(f"{SUCCESS_TAG} Saved results to {file_path}.")
    return file_path


def dump_station_results(
        df_data: pd.DataFrame, output_dir: Path, graph_name: str, method: str,
        filename_key: str = 'station_resu', verbose: bool = False
) -> str:
    # Keep only rows that are stations (not in STATS_LIST) and columns that are in METRICS_LIST:
    df_data = df_data[~df_data['Station'].isin(STATS_LIST)]
    df_data = df_data[['Station'] + METRICS_LIST]
    return save_results_to_csv(df_data, output_dir, graph_name, method, filename_key=filename_key, verbose=verbose)

In [5]:
from swissrivernetwork.benchmark.ray_evaluation import process_method
from swissrivernetwork.benchmark.util import get_run_extra_key, is_transformer_model
from typing import Union


def infer_single_run(
        graph_name: str, method: str, df_res: pd.DataFrame, settings: dict
) -> (pd.DataFrame, pd.DataFrame, dict):
    import numpy as np

    max_days = 500 if method == 'transformer_embedding' else np.inf
    # window_len = 90  # curate_window_lens(WINDOW_LENS, graph_name, mode='subsequences', max_days=max_days)
    # print(f'{INFO_TAG} Curated window lengths for graph="{graph_name}", method="{m}": {window_lens}')

    settings['path_extra_keys'] = get_run_extra_key(settings)

    df_data, extra_resu = process_method(
        graph_name, method, output_dir=OUTPUT_DIR, settings=settings, return_extra=True
    )
    new_row = {'window_len': settings['window_len']}
    for metric in METRICS_LIST:
        for stat_measure in STATS_LIST:
            col_name = f'{metric}_{stat_measure}'
            if metric in df_data.columns:
                value = df_data[df_data['Station'] == stat_measure][metric]
                if not value.empty:
                    new_row[col_name] = value.values[0]
                else:
                    new_row[col_name] = np.nan
            else:
                new_row[col_name] = np.nan
    # Add extra results:
    extra_resu_keys = [k for k in df_data.keys().to_list() if k.startswith('extra__')]
    for extra_key in extra_resu_keys:
        new_row[f'{extra_key}_Mean'] = df_data[df_data['Station'] == 'Mean'][extra_key].values[0]
    for key in extra_resu.keys():
        if f'extra__{key}' not in new_row:
            new_row[f'extra__{key}'] = extra_resu[key]

    df_res = pd.concat([df_res, pd.DataFrame([new_row])], ignore_index=True)
    return df_res, df_data, extra_resu


def infer_lstm(
        graph_name: str, method: str, df_res: pd.DataFrame, settings: dict
) -> Union[pd.DataFrame, str] | None:
    window_len = settings['window_len']
    if not df_res.empty and window_len in df_res['window_len'].values:
        print(f"{SUCCESS_TAG} Results for window_len={window_len} already exist. Skipping...")
        return None, None

    settings.update(
        {
            'positional_encoding': 'none',
        }
    )

    df_res, df_data, extra_resu = infer_single_run(graph_name, method, df_res, settings)
    file_path = save_results_to_csv(df_res, DUMP_DIR, graph_name, method, filename_key='err_90', verbose=True)
    # file_path_extra = save_results_to_json(
    #     extra_resu, DUMP_DIR, graph_name, method, filename_key='extra_resu', verbose=True
    # )
    dump_station_results(df_data, STATIONS_DUMP_DIR, graph_name, method, filename_key='station_resu', verbose=False)
    return df_res, file_path


def infer_transformer(
        graph_name: str, method: str, df_res: pd.DataFrame, settings: dict
) -> Union[pd.DataFrame, str] | None:
    window_len = settings['window_len']
    position_encodings = ['learnable', 'sinusoidal', 'rope']  # [None, 'learnable', 'sinusoidal', 'rope']
    updated = False
    for pe in position_encodings:
        print(f'{INFO_TAG} --- Processing positional_encoding={pe} ---')
        if not df_res.empty and window_len in df_res['window_len'].values and pe in df_res[
            'positional_encoding'].values:
            print(
                f'{SUCCESS_TAG} Results for window_len={window_len}, positional_encoding={pe} already exist. Skipping...'
            )
            continue

        settings.update(
            {
                'max_len': 500,
                'positional_encoding': pe,  # 'learnable' or 'sinusoidal' or 'rope' or None
            }
        )

        df_res, df_data, extra_resu = infer_single_run(graph_name, method, df_res, settings)  # Already concatenated
        if 'positional_encoding' in df_res.columns:
            df_res.loc[df_res.index[-1], 'positional_encoding'] = pe
        else:
            df_res.insert(loc=1, column='positional_encoding', value=pe)

        file_path = save_results_to_csv(df_res, DUMP_DIR, graph_name, method, filename_key='err_90', verbose=True)
        dump_station_results(
            df_data, STATIONS_DUMP_DIR, graph_name, method, filename_key=f'{pe}_station_resu',
            verbose=False
        )

        updated = True

    if not updated:
        return None, None

    return df_res, file_path


def infer_graph(graph_name: str, methods: list[str], window_len: int = 90):
    for m in methods:
        print()
        print(f'{INFO_TAG} #### Starting processing for method="{m}" on graph="{graph_name}"...')

        df_res = read_results_from_csv(DUMP_DIR, graph_name, m, filename_key='err_90')
        # extra_resu = read_results_from_json(DUMP_DIR, graph_name, m, filename_key='extra_resu')
        # print(f"{INFO_TAG} ==== Processing window_len={window_len} [{i_len + 1}/{len(window_lens)}] ====")

        settings = {
            'missing_value_method': None,  # 'mask_embedding' or 'interpolation' or 'zero' or None
            'use_current_x': True,
            'window_len': 90,
            'verbose': 1,
            'env': 'notebook',
        }

        if is_transformer_model(m):
            df_res, file_path = infer_transformer(graph_name, m, df_res, settings)
        else:
            df_res, file_path = infer_lstm(graph_name, m, df_res, settings)
        if df_res is None:
            continue

        print(f'{SUCCESS_TAG} Completed processing for window_len={window_len}. Results saved to {file_path}. ')

        print(f'{SUCCESS_TAG} #### Finished processing for method="{m}" on graph="{graph_name}"!')

In [6]:
GRAPH_NAMES = ['swiss-1990', 'swiss-2010', 'zurich']
METHODS = ['lstm', 'transformer', 'graphlet', 'transformer_graphlet', 'lstm_embedding', 'transformer_embedding',
           'stgnn', 'transformer_stgnn']

In [7]:
# Compute results and save to CSV:
for graph_name in GRAPH_NAMES[0:0]:
    print(f'{INFO_TAG} ================= Starting processing for graph="{graph_name}" =================')
    infer_graph(graph_name, METHODS[0:], window_len=90)  # test [2:3]

# Plot radar charts

## Gather data

### Gather Err 90 data, Model Size data, Runtime data

In [8]:
GRAPH_NAMES = ['swiss-1990', 'swiss-2010', 'zurich']
METHODS = ['lstm', 'transformer', 'graphlet', 'transformer_graphlet', 'lstm_embedding', 'transformer_embedding',
           'stgnn', 'transformer_stgnn']

In [9]:
import pandas as pd


def get_err_from_df_lstm(df_res: pd.DataFrame, metric: str, window_len: int = 90) -> float | None:
    # Filter rows with window_len=window_len
    df_filtered = df_res[df_res['window_len'] == window_len]
    if df_filtered.empty:
        print(f'{ISSUE_TAG} No data found for window_len={window_len}.')
        return None
    if len(df_filtered) > 1:
        raise ValueError(f'{ISSUE_TAG} Multiple rows found for window_len={window_len}. Expected only one.')
    return float(df_filtered.iloc[0][metric])


def get_err_from_df_transformer(df_res: pd.DataFrame, metric: str, window_len: int = 90) -> (float | None, str | None):
    # Filter rows with window_len=window_len
    df_filtered = df_res[df_res['window_len'] == window_len]
    if df_filtered.empty:
        print(f'{ISSUE_TAG} No data found for window_len={window_len}.')
        return None, None
    # Get the row with the best metric (lowest for RMSE and MAE, highest for NSE):
    if metric.startswith('RMSE') or metric.startswith('MAE'):
        best_row = df_filtered.loc[df_filtered[metric].idxmin()]
    elif metric.startswith('NSE'):
        best_row = df_filtered.loc[df_filtered[metric].idxmax()]
    elif any([k in metric for k in ['model_size', 'train_time', 'infer_time']]):
        best_row = df_filtered.loc[df_filtered['RMSE_Mean'].idxmin()]  # Choose the one with min RMSE Mean
    else:
        print(f'{ISSUE_TAG} Unknown metric "{metric}".')
        return None, None
    return float(best_row[metric]), best_row['positional_encoding']


def gather_err_data(
        graph_names: list[str], methods: list[str], metric: str = 'RMSE_Mean', window_len: int = 90
) -> dict:
    data, best_pes = {}, {}
    for graph_name in graph_names:
        data[graph_name] = {}
        best_pes[graph_name] = {}
        for method in methods:
            df_res = read_results_from_csv(DUMP_DIR, graph_name, method, filename_key='err_90')
            if df_res.empty:
                print(f"{ISSUE_TAG} No data found for graph='{graph_name}', method='{method}'. Skipping...")
                continue
            if is_transformer_model(method):
                result, best_pe = get_err_from_df_transformer(df_res, metric, window_len=window_len)
                best_pes[graph_name][method] = best_pe
            else:
                result = get_err_from_df_lstm(df_res, metric, window_len=window_len)
            print(f'{SUCCESS_TAG} Gathered {metric}={result} for graph="{graph_name}", method="{method}".')

            data[graph_name][method] = result

    return data, best_pes


def normalize_err_data(
        err_data: dict, min_is_better: bool = True, global_min: float | None = None, global_max: float | None = None,
) -> dict:
    """
    Normalize error data to [0, 1] range across all graphs and methods. Higher values are better.
    """
    # Find global min and max
    return err_data
    all_values = []
    for graph_name in err_data:
        for method in err_data[graph_name]:
            value = err_data[graph_name][method]
            if value is not None:
                all_values.append(value)
    global_min = min(all_values) if global_min is None else global_min
    global_max = max(all_values) if global_max is None else global_max

    # Normalize data
    normalized_data = {}
    for graph_name in err_data:
        normalized_data[graph_name] = {}
        for method in err_data[graph_name]:
            value = err_data[graph_name][method]
            if value is None:
                normalized_value = None
            else:
                if min_is_better:
                    normalized_value = (global_max - value) / (global_max - global_min)  # Higher is better
                else:
                    normalized_value = (value - global_min) / (global_max - global_min)  # Higher is better
            normalized_data[graph_name][method] = normalized_value

    return normalized_data

In [10]:
metric = 'RMSE_Mean'
err_90_data, rmse_best_pes = gather_err_data(GRAPH_NAMES, METHODS, metric, window_len=90)
normalized_err_90_data = normalize_err_data(err_90_data, min_is_better=(not metric.startswith('NSE')))
print(normalized_err_90_data)

In [11]:
metric = 'MAE_Mean'
mae_data, mae_best_pes = gather_err_data(GRAPH_NAMES, METHODS, metric, window_len=90)
normalized_mae_data = normalize_err_data(mae_data, min_is_better=True)
print(normalized_mae_data)

In [12]:
metric = 'NSE_Mean'
nse_data, nse_best_pes = gather_err_data(GRAPH_NAMES, METHODS, metric, window_len=90)
normalized_nse_data = nse_data  # normalize_err_data(nse_data, min_is_better=False, global_min=0.0, global_max=1.0)
print(normalized_nse_data)

### Gather forecasting results (RMSE, MAE, NSE), window length results, and noise robustness results:
**Forecasting results are loaded from files saved by running `future_steps_resu.ipynb` in the same directory.**

**Window length results are loaded from files saved by running `window_length_resu.ipynb` in the same directory.**

**Noise robustness results are loaded from files saved by running `noise_robustness_resu.ipynb` in the same directory.**

In [13]:
from typing import Any


def aggregate_fs_metric(values: dict[Any, float]) -> float:
    values = list(values.values())
    return sum(values) / len(values)


def aggregate_win_len_metric(values: dict[Any, float]) -> float:
    """
    $\text{WT-Score}=\frac{\sum_{i} w(c_i) M(c_i)}{\sum_{i} w(c_i)}$,
    where $w(l)=2^{-l/h_{1/2}}$.
    """
    keys = list(values.keys())
    max_len = max(keys)
    h_half = max_len // 2
    weighted_sum = 0.0
    weight_total = 0.0
    for l in keys:
        w_l = 2 ** (-l / h_half)
        weighted_sum += w_l * values[l]
        weight_total += w_l
    return weighted_sum / weight_total


def aggregate_noise_metric(values: dict[Any, float]) -> float:
    """
    $\text{WT-Score}=\frac{\sum_{i} w(c_i) M(c_i)}{\sum_{i} w(c_i)}$,
    where $w(\sigma)=\sigma^\gamma$, and $\sigma$ is the noise level related to the standard deviation of the baseline, and $\gamma=2$.
    """
    keys = list(values.keys())
    gamma = 2.0
    weighted_sum = 0.0
    weight_total = 0.0
    for sigma in keys:
        w_sigma = sigma ** gamma
        weighted_sum += w_sigma * values[sigma]
        weight_total += w_sigma
    return weighted_sum / weight_total


<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:27: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:27: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_194575/641893525.py:10: SyntaxWarning: invalid escape sequence '\s'
  """
/tmp/ipykernel_194575/641893525.py:27: SyntaxWarning: invalid escape sequence '\s'
  """


In [14]:
from typing import Callable
import pandas as pd


def get_metric_along_variable(
        df_res: pd.DataFrame, metric: str, variable_key: str, variable_list: list,
):
    results = {}
    for var_val in variable_list:
        # Filter rows with variable_key=var_val:
        df_filtered = df_res[df_res[variable_key] == var_val]
        if df_filtered.empty:
            print(f'{ISSUE_TAG} No data found for {variable_key}={var_val}.')
            return None
        if len(df_filtered) > 1:
            raise ValueError(f'{ISSUE_TAG} Multiple rows found for {variable_key}={var_val}. Expected only one.')
        results[var_val] = float(df_filtered.iloc[0][metric])
    return results


def get_variable_data_from_df_lstm(
        df_res: pd.DataFrame, metric: str, variable_key: str, variable_list: list, aggregate_fun: Callable
) -> float | None:
    results = get_metric_along_variable(df_res, metric, variable_key=variable_key, variable_list=variable_list)
    if results is None:
        return None
    return aggregate_fun(results)


def get_variable_data_from_df_transformer(
        df_res: pd.DataFrame, metric: str, variable_key: str, variable_list: list, aggregate_fun: Callable
) -> float | None:
    import numpy as np

    positional_encodings = ['learnable', 'sinusoidal', 'rope']
    results = []
    for pe in positional_encodings:
        # Filter rows with positional_encoding=pe:
        df_pe = df_res[df_res['positional_encoding'] == pe]
        if df_pe.empty:
            print(f'{ISSUE_TAG} No data found for positional_encoding={pe}.')
            return None
        pe_results = get_metric_along_variable(df_pe, metric, variable_key=variable_key, variable_list=variable_list)
        if pe_results is None:
            return None
        pe_results = aggregate_fun(pe_results)
        results.append(pe_results)

    # Get the results with the best metric (lowest for RMSE and MAE, highest for NSE):
    if metric.startswith('RMSE') or metric.startswith('MAE'):
        best_idx = np.argmin(results)
        best_result = results[best_idx]
        best_pe = positional_encodings[best_idx]
    elif metric.startswith('NSE'):
        best_idx = np.argmax(results)
        best_result = results[best_idx]
        best_pe = positional_encodings[best_idx]
    # elif any([k in metric for k in ['model_size', 'train_time', 'infer_time']]):
    #     best_row = df_filtered.loc[df_filtered['RMSE_Mean'].idxmin()]  # Choose the one with min RMSE Mean
    else:
        print(f'{ISSUE_TAG} Unknown metric "{metric}".')
        return None, None
    return best_result, best_pe


def gather_variable_data(
        graph_names: list[str], methods: list[str], metric: str, variable_key: str, variable_list: list,
        aggregate_fun: Callable, filename_key: str, dump_dir: Path = DUMP_DIR
) -> (dict, dict):
    data, best_pes = {}, {}
    for graph_name in graph_names:
        data[graph_name] = {}
        best_pes[graph_name] = {}
        for method in methods:
            df_res = read_results_from_csv(dump_dir, graph_name, method, filename_key=filename_key)
            if df_res.empty:
                print(f"{ISSUE_TAG} No data found for graph='{graph_name}', method='{method}'. Skipping...")
                continue
            if is_transformer_model(method):
                result, best_pe = get_variable_data_from_df_transformer(
                    df_res, metric, variable_key, variable_list, aggregate_fun
                )
                best_pes[graph_name][method] = best_pe
            else:
                result = get_variable_data_from_df_lstm(df_res, metric, variable_key, variable_list, aggregate_fun)
            print(f'{SUCCESS_TAG} Gathered {metric}={result} for graph="{graph_name}", method="{method}".')

            data[graph_name][method] = result

    return data, best_pes

#### Forecasting results:

In [15]:
FS_DUMP_DIR = (CUR_ABS_DIR / 'outputs/future_steps/').resolve()
filename_key = 'fs_resu'
MAX_HORIZON = 7
variable_key = 'future_step'
variable_list = list(range(1, MAX_HORIZON + 1))
aggregate_fun = aggregate_fs_metric

In [16]:
metric = 'RMSE_Mean'
rmse_f_data, rmse_f_best_pes = gather_variable_data(
    GRAPH_NAMES, METHODS, metric, variable_key, variable_list, aggregate_fun, filename_key, FS_DUMP_DIR
)
normalized_rmse_f_data = normalize_err_data(rmse_f_data, min_is_better=(not metric.startswith('NSE')))
print(normalized_rmse_f_data)

In [17]:
metric = 'MAE_Mean'
mae_f_data, mae_f_best_pes = gather_variable_data(
    GRAPH_NAMES, METHODS, metric, variable_key, variable_list, aggregate_fun, filename_key, FS_DUMP_DIR
)
normalized_mae_f_data = normalize_err_data(mae_f_data, min_is_better=(not metric.startswith('NSE')))
print(normalized_mae_f_data)

In [18]:
metric = 'NSE_Mean'
nse_f_data, nse_f_best_pes = gather_variable_data(
    GRAPH_NAMES, METHODS, metric, variable_key, variable_list, aggregate_fun, filename_key, FS_DUMP_DIR
)
normalized_nse_f_data = normalize_err_data(nse_f_data, min_is_better=(not metric.startswith('NSE')))
print(normalized_nse_f_data)

### Combine all data into a single structure to construct latex table:

In [19]:
data = [[]]
for graph_name in GRAPH_NAMES:
    data += [
        (f'{graph_name}, Isolated', [
            [],
            [],
        ]),
        (f'{graph_name}, Graphlet', [
            [],
            [],
        ]),
        (f'{graph_name}, Embedded', [
            [],
            [],
        ]),
        (f'{graph_name}, ST-GNN', [
            [],
            [],
        ]),
    ]


def append_data_to_visual(data: list, data_to_add: dict, varlabel_to_add: str) -> list:
    data[0].append(varlabel_to_add)
    n_models = 4
    for idx_g, graph_name in enumerate(GRAPH_NAMES):
        data[1 + n_models * idx_g][1][0].append(data_to_add[graph_name].get('lstm'))
        data[1 + n_models * idx_g][1][1].append(data_to_add[graph_name].get('transformer'))
        data[2 + n_models * idx_g][1][0].append(data_to_add[graph_name].get('graphlet'))
        data[2 + n_models * idx_g][1][1].append(data_to_add[graph_name].get('transformer_graphlet'))
        data[3 + n_models * idx_g][1][0].append(data_to_add[graph_name].get('lstm_embedding'))
        data[3 + n_models * idx_g][1][1].append(data_to_add[graph_name].get('transformer_embedding'))
        data[4 + n_models * idx_g][1][0].append(data_to_add[graph_name].get('stgnn'))
        data[4 + n_models * idx_g][1][1].append(data_to_add[graph_name].get('transformer_stgnn'))
    return data

In [20]:
# Metrics for nowcasting:
data = append_data_to_visual(data, normalized_err_90_data, 'RMSE/N')
data = append_data_to_visual(data, normalized_mae_data, 'MAE/N')
data = append_data_to_visual(data, normalized_nse_data, 'NSE/N')
# Metrics for forecasting:
data = append_data_to_visual(data, normalized_rmse_f_data, 'RMSE/F')
data = append_data_to_visual(data, normalized_mae_f_data, 'MAE/F')
data = append_data_to_visual(data, normalized_nse_f_data, 'NSE/F')
print(len(data))
print(data)

### Compute significant difference using Wilcoxon signed-rank test:

In [21]:
from scipy.stats import wilcoxon

ARCHITECTURES = ['Isolated', 'Graphlet', 'Embedded', 'ST-GNN']
archs_to_methods_dict = {
    'Isolated': ['lstm', 'transformer'],
    'Graphlet': ['graphlet', 'transformer_graphlet'],
    'Embedded': ['lstm_embedding', 'transformer_embedding'],
    'ST-GNN': ['stgnn', 'transformer_stgnn'],
}
metric_to_best_pe_dict = {
    'nowcasting': {
        'RMSE': rmse_best_pes,
        'MAE': mae_best_pes,
        'NSE': nse_best_pes,
    },
    'forecasting': {
        'RMSE': rmse_f_best_pes,
        'MAE': mae_f_best_pes,
        'NSE': nse_f_best_pes,
    }
}

significant_diff = {'nowcasting': {}, 'forecasting': {}}
for i_graph, graph_name in enumerate(GRAPH_NAMES):
    for casting in ['nowcasting', 'forecasting']:
        significant_diff[casting][graph_name] = {}
        for i_arch, arch in enumerate(ARCHITECTURES):
            significant_diff[casting][graph_name][arch] = {}
            lstm_method = archs_to_methods_dict[arch][0]
            transformer_method = archs_to_methods_dict[arch][1]
            for metric in ['RMSE', 'MAE', 'NSE']:
                # Get LSTM reference station results:
                cur_df_ref = read_results_from_csv(
                    STATIONS_DUMP_DIR, graph_name, lstm_method,
                    filename_key='station_resu_f' if casting == 'forecasting' else 'station_resu'
                )
                cur_ref = cur_df_ref[['Station', metric]]
                cur_best_pe = metric_to_best_pe_dict[casting][metric][graph_name][transformer_method]
                cur_df = read_results_from_csv(
                    STATIONS_DUMP_DIR, graph_name, transformer_method,
                    filename_key=f'{cur_best_pe}_station_resu_f' if casting == 'forecasting' else f'{cur_best_pe}_station_resu'
                )
                # Get columns "Station" and metric:
                cur_evals = cur_df[['Station', metric]]
                # Merge on "Station":
                merged_df = pd.merge(cur_ref, cur_evals, on='Station', suffixes=('_lstm', '_transformer'))
                # Prepare data for Wilcoxon test:
                lstm_values = merged_df[f'{metric}_lstm'].tolist()
                transformer_values = merged_df[f'{metric}_transformer'].tolist()

                if len(lstm_values) < 2:
                    print(
                        f'{ISSUE_TAG} Not enough data to perform Wilcoxon test for graph="{graph_name}", arch="{arch}", '
                        f'metric="{metric}" (nowcasting).'
                    )
                    significant_diff[casting][graph_name][arch][metric] = False
                    continue

                stat, p_value = wilcoxon(lstm_values, transformer_values)
                alpha = 0.05
                is_significant = p_value < alpha
                significant_diff[casting][graph_name][arch][metric] = is_significant
                print(
                    f'{INFO_TAG} Wilcoxon test for graph="{graph_name}", arch="{arch}", metric="{metric}" ({casting}): '
                    f'stat={stat}, p-value={p_value}, significant={is_significant}'
                )

### Construct LaTeX table:

In [22]:
GRAPH_NAMES = ['swiss-1990', 'swiss-2010', 'zurich']
# METHODS = ['lstm', 'transformer', 'graphlet', 'transformer_graphlet', 'lstm_embedding', 'transformer_embedding',
#            'stgnn', 'transformer_stgnn']
ARCHITECTURES = ['Isolated', 'Graphlet', 'Embedded', 'ST-GNN']

In [24]:
import numpy as np


def get_value_str_suffix(change_in_percent: float, larger_is_better: bool = False, threshold: float = 0.05) -> str:
    if np.isnan(change_in_percent):
        return ''
    condition_better = (change_in_percent >= threshold) if larger_is_better else (change_in_percent <= -threshold)
    condition_worse = (change_in_percent <= -threshold) if larger_is_better else (change_in_percent >= threshold)
    if condition_better:
        str_color = 'Green'
        str_arrow = '$\\uparrow$'
    elif condition_worse:
        str_color = 'Red'
        str_arrow = '$\\downarrow$'
    else:
        str_color = 'Blue'
        str_arrow = ''
    return f'\\textcolor{{{str_color}}}{{\\scriptsize{{{abs(change_in_percent):.1f}\\%{str_arrow}}}}}~'


def get_best_performance(data: list) -> float | None:
    idx_best_perf = []
    for i_graph, graph_name in enumerate(GRAPH_NAMES):
        value_lists = []
        for i_model, model_name in enumerate(['LSTM', 'Transformer']):
            for i_arch, arch in enumerate(ARCHITECTURES):
                # RMSE, MAE, NSE for nowcasting and forecasting:
                all_values = np.array(data[1 + 4 * i_graph + i_arch][1][i_model][0:6])
                value_lists.append(all_values)
        value_array = np.array(value_lists)
        # print(f'Graph: {graph_name}')
        # print(value_array)
        max_i_vals = np.argmax(value_array, axis=0)
        min_i_vals = np.argmin(value_array, axis=0)
        best_i_vals = [max_i if i in [2, 5] else min_i for i, (min_i, max_i) in
                       enumerate(zip(min_i_vals, max_i_vals))]
        # print(max_i_vals)
        # print(min_i_vals)
        # print(best_i_vals)
        idx_best_perf.append(best_i_vals)
    return idx_best_perf


def is_best_performance(idx_best_perf: list, i_graph: int, i_model: int, i_arch: int, i_metric: int) -> bool:
    n_archs = len(ARCHITECTURES)
    idx = i_model * n_archs + i_arch
    return idx_best_perf[i_graph][i_metric] == idx


# if __name__ == '__main__':
idx_best_perf = get_best_performance(data)

latex_str = ''
for i_graph, graph_name in enumerate(GRAPH_NAMES):
    latex_str += f'\\bf {graph_name.capitalize()} & '
    for i_model, model_name in enumerate(['LSTM', 'Transformer']):
        if i_model == 1:
            latex_str += ' & '
        latex_str += f'{model_name} & '
        for i_arch, arch in enumerate(ARCHITECTURES):
            if i_model == 0 and i_arch == 0:
                pass
            elif i_model == 1 and i_arch == 0:
                pass
            else:
                latex_str += ' & & '
            latex_str += f'{arch} & '
            all_values = np.array(
                data[1 + 4 * i_graph + i_arch][1][i_model][0:6]
            )  # RMSE, MAE, NSE for nowcasting and forecasting

            # Get the value change:
            if i_model == 1:
                all_values_lstm = np.array(data[1 + 4 * i_graph + i_arch][1][0][0:6])  # LSTM values
                nan_lstm = np.where(all_values_lstm == None, np.nan, all_values_lstm)
                nan_transformer = np.where(all_values == None, np.nan, all_values)
                changes_in_percent = ((nan_transformer - nan_lstm) / nan_lstm) * 100.0

            for i_val, val in enumerate(all_values):
                if val is None:
                    latex_str += '-'
                else:
                    formatted_val = (lambda x: f"{x:.3f}".lstrip("0").replace("-0", "-"))(val)
                    if is_best_performance(idx_best_perf, i_graph, i_model, i_arch, i_val):
                        formatted_val = f'\\textbf{{{formatted_val}}}'
                        formatted_val = f'\\underline{{{formatted_val}}}'
                    larger_is_better = True if i_val in [2, 5] else False  # NSE larger is better
                    latex_str += (get_value_str_suffix(
                        changes_in_percent[i_val], larger_is_better=larger_is_better
                    ) if i_model == 1 else '') + formatted_val

                # Add a star to indicate significant difference:
                if i_model == 1:  # Transformer models only
                    arch_name = ARCHITECTURES[i_arch]
                    if_sig_diff = significant_diff.get(
                        'nowcasting' if i_val < 3 else 'forecasting', {}
                    ).get(graph_name, {}).get(arch_name, {}).get(
                        'RMSE' if i_val % 3 == 0 else 'MAE' if i_val % 3 == 1 else 'NSE', None
                    )
                    if if_sig_diff is None:
                        print(
                            f'{ISSUE_TAG} Missing significance test result for graph="{graph_name}", arch="{arch_name}", '
                            f'metric="{"RMSE" if i_val % 3 == 0 else "MAE" if i_val % 3 == 1 else "NSE"}" '
                            f'({"nowcasting" if i_val < 3 else "forecasting"}).'
                        )
                    elif if_sig_diff:
                        latex_str += '$^{*}$'

                if i_val < len(all_values) - 1:
                    latex_str += ' & '
            if i_arch < len(ARCHITECTURES) - 1:
                latex_str += ' \\\\ \n'

        if i_model < 1:
            latex_str += ' \\\\ \n \\cmidrule(r){2-9} \n'

    if i_graph < len(GRAPH_NAMES) - 1:
        latex_str += ' \\\\ \n\\midrule \n'

latex_str += ' \\\\'

# print(latex_str)  # directly print does not display correctly in Jupyter Notebook
from IPython.display import HTML

HTML(f"<pre>{latex_str}</pre>")
